In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import libraries
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import transformers
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast
device = 'mps'

In [3]:
# Import pretrained tokenizer and model from https://huggingface.co/maymuni/bert-base-turkish-cased-emotion-analysis?text=I+like+you.+I+love+you
tokenizer = AutoTokenizer.from_pretrained("maymuni/bert-base-turkish-cased-emotion-analysis")
bert = AutoModel.from_pretrained("maymuni/bert-base-turkish-cased-emotion-analysis",return_dict=False)

/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [5]:
class Arch(nn.Module):

    def __init__(self, bert):
      super(Arch, self).__init__()
      self.bert = bert 
      self.dropout = nn.Dropout(0.1)
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)
      self.fc3 = nn.Linear(512,9)
      self.softmax = nn.LogSoftmax(dim=1)
      
    def forward(self, sent_id, mask):
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc3(x)
      x = self.softmax(x)
      return x

In [6]:
device = torch.device('mps')

# pass the pre-trained model to our define architecture
model = Arch(bert)

# push the model to GPU
model = model.to(device)

In [7]:

# model_Final_son.pt modeli yukle ve test et
model.load_state_dict(torch.load('my_model_final.pt', map_location=torch.device('mps')))


<All keys matched successfully>

In [8]:
import heapq

#Define predict function
def predict_emotion(text):
    tokenized = tokenizer.encode_plus(
        text,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    input_ids = tokenized['input_ids']
    attention_mask = tokenized['attention_mask']

    seq = torch.tensor(input_ids)
    mask = torch.tensor(attention_mask)
    seq = seq.unsqueeze(0)
    mask = mask.unsqueeze(0)
    preds = model(seq.to(device), mask.to(device))
    preds = preds.detach().cpu().numpy()
    preds = torch.tensor(preds)
    probabilities = nn.functional.softmax(preds)

    # Get the top 3 emotions by probability
    top3_emotions = heapq.nlargest(3, enumerate(probabilities[0]), key=lambda x: x[1])

    # Get the names and probabilities of the top 3 emotions
    top3_emotion_names = ['kızgın', 'üzgün', 'korku', 'iğrenme', 'mutlu', 'aşk', 'merak', 'utanç', 'şaşkınlık']
    top3_emotion_probabilities = {top3_emotion_names[index]: float(prob) for index, prob in top3_emotions}

    return top3_emotion_probabilities

In [9]:
predict_emotion('içimde bir endişe var')

/opt/homebrew/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/var/folders/c3/dqbzx13n0p59hyktphf9rw540000gn/T/ipykernel_27678/1024367870.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = nn.functional.softmax(preds)


{'iğrenme': 0.9926135540008545,
 'utanç': 0.004823864437639713,
 'korku': 0.0008097006939351559}